In [1]:
from util import go_to_project_root
go_to_project_root()

CWD cambiado a la raíz del proyecto: /home/epsilon/Documents/doctorado/S2/ENSTA630/tareas/HW2


# Problema 1

In [2]:
import numpy as np
from source.data.loaders import create_dataset
from source.models.var import VAR

%load_ext autoreload
%autoreload 2

### Cargar datos

In [27]:
path = './data/base_25.xls'
dataset = create_dataset(path, problem=5)

[INFO] ¡Datos cargados exitosamente!


In [28]:
i_reshaped = dataset['i'][12:]
X = np.vstack([dataset['pi'], dataset['y'], i_reshaped]).T

In [ ]:
n = 3 # numero de variables
p = 2 # rezagos

model = VAR(inp_dim=n, 
            c = 0,
            p=p, 
            params_distribution='stationary', low=0, high=1)
model._is_stationary()

Repitiendo el valor de c


np.True_